In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract

spark = SparkSession.builder.appName("Nasa Log File Analysis").getOrCreate()
logs_df = spark.read.text("dbfs:/FileStore/shared_uploads/chitengamarvellous@gmail.com/NASA_access_log_Jul95.gz")
logs_df.show(5, truncate = False)

logs_df_parsed = logs_df.select(
    regexp_extract('value', r'^(\S+)', 1).alias('Ip'),
    regexp_extract('value', r'\[(.*?)\]', 1).alias('timestamp'),
    regexp_extract('value', r'\"(S+)', 1).alias('method'),
    regexp_extract('value',r'\"(?:\S+)\s(\S+)', 1).alias('Url'),
    regexp_extract('value',r'\"\s(\d{3})' ,1).alias('Status_Code'),
    regexp_extract('value',r'\"\s\d{3}\s(\d+|-)' ,1).alias('Size In Bytes')
)
logs_df_parsed.show(5, truncate = False)

logs_df_parsed.groupBy("Url").count().orderBy("count", ascending=False).show(10, truncate=False)
logs_df_parsed.groupBy("Ip").count().orderBy("count", ascending=False).show(10)
logs_df_parsed.count()
logs_df_parsed.filter(logs_df_parsed.Status_Code != '200').count()


+-----------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                  |
+-----------------------------------------------------------------------------------------------------------------------+
|199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245                                 |
|unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985                      |
|199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085   |
|burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0               |
|199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179|
+-----------------------